In [5]:
import sqlalchemy
import pandas as pd
import numpy as np
import sys
import mysql.connector
import plotly_express as px

In [ ]:
database_username = '***'
database_password = '***'
database_ip       = 'localhost'
database_name     = 'Projet_data_eng'

database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [7]:
conso_elec = pd.read_sql("SELECT * FROM Conso_elec ",database_connection)
pib = pd.read_sql("SELECT * FROM PIB ",database_connection)
emission = pd.read_sql("SELECT * FROM Emission_CO2 ",database_connection)
pib_hbt = pd.read_sql("SELECT * FROM Pib_hbt ",database_connection)
pop_active = pd.read_sql("SELECT * FROM Pop_active ",database_connection)
pop_active_femme = pd.read_sql("SELECT * FROM Pourcentage_femmes_actives ",database_connection)

In [8]:
data = conso_elec[["Country Name","Country Code","year"]]

In [9]:
emission_par_conso = emission["Emission_CO2_tonnes_hbt"] * 1000 / conso_elec["Conso_elec_KWh_hbt"]
emission_par_conso_arrondi=[]
for indicateur in list(emission_par_conso):
    indicateur=float("{:.2f}".format(indicateur))
    emission_par_conso_arrondi.append(indicateur)


data = data.assign(Emission_sur_conso=emission_par_conso_arrondi)

In [10]:
pib_par_active = pib["Pib_dollars"] / pop_active["Population_active"]
pib_par_active_arrondi=[]
pib_par_hbt_arrondi=[]
for indicateur in list(pib_par_active):
    indicateur=float("{:.2f}".format(indicateur))
    pib_par_active_arrondi.append(indicateur)
    
for indicateur in list(pib_hbt["Pib_hbt"]):
    indicateur=float("{:.2f}".format(indicateur))
    pib_par_hbt_arrondi.append(indicateur)

data = data.assign(PIB_sur_Pop_Active=pib_par_active_arrondi)
data = data.assign(PIB_hbt=pib_par_hbt_arrondi)

In [11]:
population_totale = pib["Pib_dollars"] // pib_hbt["Pib_hbt"]
pourcentage_pop_active = pop_active["Population_active"] * 100 / population_totale
pourcentage_pop_active_arrondi=[]
pourcentage_femme_active_arrondi=[]
pourcentage_homme_active_arrondi=[]

for indicateur in list(pourcentage_pop_active):
    indicateur=float("{:.2f}".format(indicateur))
    pourcentage_pop_active_arrondi.append(indicateur)
    
for indicateur in list(pop_active_femme["Pourcentage_femme_active"]):
    indicateur=float("{:.2f}".format(indicateur))
    pourcentage_femme_active_arrondi.append(indicateur)
    
for pourcentage in pourcentage_femme_active_arrondi:
    pourcentage_homme_active_arrondi.append(100-pourcentage)

data = data.assign(Pourcentage_pop_active=pourcentage_pop_active_arrondi)
data = data.assign(Pourcentage_femme_active=pourcentage_femme_active_arrondi)
data = data.assign(Pourcentage_homme_active=pourcentage_homme_active_arrondi)

In [12]:
data.to_sql(con=database_connection, name='indicateurs_ESG', if_exists='replace')

-1

# Analyse

In [56]:
data2021 = data[(data["year"]=='2021')]

top_pays = data2021.sort_values(by=['Pourcentage_pop_active'],ascending=False)
top_pays = top_pays[["Country Name", "Country Code","Pourcentage_pop_active"]]
top_pays = top_pays.dropna()

px.bar(top_pays.head(10),x="Country Name",y="Pourcentage_pop_active",
       title="Top 10 des pays avec les pourcentages de population active les plus eleves en 2021")

In [14]:
liste_indicateurs = []
liste_annees = []
liste_sexes = []

for year in list(pd.unique(data.year)):
    moyenne_femme_active = np.mean(data[data.year==year]["Pourcentage_femme_active"])
    moyenne_homme_active = np.mean(data[data.year==year]["Pourcentage_homme_active"])
    liste_annees.extend([year,year])
    liste_sexes.extend(["Femme","Homme"])
    liste_indicateurs.extend([moyenne_femme_active,moyenne_homme_active])
    
repartition_hf = pd.DataFrame({"Year":liste_annees,
                               "Sexe":liste_sexes,
                               "Pourcentage":liste_indicateurs})

px.bar(
    repartition_hf,
    x="Year",
    y="Pourcentage",
    color="Sexe",
    title="Evolution de la repartition des hommes et des femmes au fil des annees dans le monde"
)

In [55]:
top_pays_active = data2021.sort_values(by=['PIB_sur_Pop_Active'],ascending=False)
top_pays_active = top_pays_active[["Country Name", "Country Code","PIB_sur_Pop_Active"]]
top_pays_active = top_pays_active.dropna()

px.bar(top_pays_active.head(10),x="Country Name",y="PIB_sur_Pop_Active",
       title="Top 10 des pays avec les PIB/population_active les plus eleves en 2021")

In [57]:
liste_indicateurs = []
liste_em = []
liste_conso = []
liste_annees = []

for year in list(pd.unique(data.year)):
    moyenne_em_sur_conso = np.mean(data[data.year==year]["Emission_sur_conso"])
    em = np.mean(emission[emission.year==year]["Emission_CO2_tonnes_hbt"])
    em = em * 1000
    conso = np.mean(conso_elec[conso_elec.year==year]["Conso_elec_KWh_hbt"])
    liste_annees.append(year)
    liste_indicateurs.append(moyenne_em_sur_conso)
    liste_em.append(em)
    liste_conso.append(conso)
    
tab_em_sur_conso = pd.DataFrame({"Year":liste_annees,
                               "Emission_sur_conso":liste_indicateurs,
                               "Emission_CO2":liste_em,
                               "Conso_electricite":liste_conso})

tab_em_sur_conso = tab_em_sur_conso.dropna()

px.line(
    tab_em_sur_conso,
    x="Year",
    y=["Emission_CO2","Conso_electricite"],
    title="Evolution des emissions de CO2 et de la conso d'electricite au fil des annees dans le monde",
    labels={"Year": "Annee"}
)

In [17]:
px.line(
    tab_em_sur_conso,
    x="Year",
    y="Emission_sur_conso",
    title="Evolution des emissions de CO2 par rapport a la conso d'electricite au fil des annees dans le monde",
    labels={"Year": "Annee"}
)